In [1]:
#Import Libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

## Importing Dataframes

In [2]:
path=r'C:\Users\solom\Desktop\Instacart Basket Analysis'

In [3]:
df_ords_prods = pd.read_pickle(os.path.join(path, '02 Data', 'Manipulated Data', 'ords_prods_grouped_aggregate.pkl'))

In [4]:
df_cust=pd.read_csv(os.path.join(path,'02 Data', 'Original Data', 'customers.csv'))

In [5]:
df_cust.head()

user_id First Name    Surnam  Gender       STATE  Age date_joined  \
0    26711    Deborah  Esquivel  Female    Missouri   48    1/1/2017   
1    33890   Patricia      Hart  Female  New Mexico   36    1/1/2017   
2    65803    Kenneth    Farley    Male       Idaho   35    1/1/2017   
3   125935   Michelle     Hicks  Female        Iowa   40    1/1/2017   
4   130797        Ann   Gilmore  Female    Maryland   26    1/1/2017   

   n_dependants fam_status  income  
0             3    married  165665  
1             0     single   59285  
2             2    married   99568  
3             0     single   42049  
4             1    married   40374

## Wrangling data

In [6]:
# Rename columns
df_cust.rename(columns={'Surnam':'surname', 'fam_status':'family_status','STATE':'state','n_dependants':'number_of_dependants',
                  'First Name':'first_name'}, inplace=True)

In [7]:
df_cust.head()

user_id first_name   surname  Gender       state  Age date_joined  \
0    26711    Deborah  Esquivel  Female    Missouri   48    1/1/2017   
1    33890   Patricia      Hart  Female  New Mexico   36    1/1/2017   
2    65803    Kenneth    Farley    Male       Idaho   35    1/1/2017   
3   125935   Michelle     Hicks  Female        Iowa   40    1/1/2017   
4   130797        Ann   Gilmore  Female    Maryland   26    1/1/2017   

   number_of_dependants family_status  income  
0                     3       married  165665  
1                     0        single   59285  
2                     2       married   99568  
3                     0        single   42049  
4                     1       married   40374

## Data quality and consistency checks
### Missing Values

In [8]:
df_cust.isnull().sum()

user_id                     0
first_name              11259
surname                     0
Gender                      0
state                       0
Age                         0
date_joined                 0
number_of_dependants        0
family_status               0
income                      0
dtype: int64

In [9]:
df_cust[df_cust['first_name'].isnull() == True]

user_id first_name   surname  Gender         state  Age date_joined  \
53        76659        NaN   Gilbert    Male      Colorado   26    1/1/2017   
73        13738        NaN     Frost  Female     Louisiana   39    1/1/2017   
82        89996        NaN    Dawson  Female        Oregon   52    1/1/2017   
99        96166        NaN   Oconnor    Male      Oklahoma   51    1/1/2017   
105       29778        NaN    Dawson  Female          Utah   63    1/1/2017   
...         ...        ...       ...     ...           ...  ...         ...   
206038   121317        NaN    Melton    Male  Pennsylvania   28   3/31/2020   
206044   200799        NaN  Copeland  Female        Hawaii   52    4/1/2020   
206090   167394        NaN     Frost  Female        Hawaii   61    4/1/2020   
206162   187532        NaN     Floyd  Female    California   39    4/1/2020   
206171   116898        NaN   Delgado  Female      Colorado   23    4/1/2020   

        number_of_dependants family_status  income  
53                         2       married   41709  
73                         0        single   82518  
82                         3       married  117099  
99                         1       married  155673  
105                        3       married  151819  
...                      ...           ...     ...  
206038                     3       married   87783  
206044                     2       married  108488  
206090                     1       married   45275  
206162                     0        single   56325  
206171                     2       married   59222  

[11259 rows x 10 columns]

### There were 11259 missing values in the first name column but they shouldn't be dropped since all rows hold useful information

### Duplicate rows

In [10]:
df_cust_dups= df_cust[df_cust.duplicated()]

In [11]:
df_cust_dups

Empty DataFrame
Columns: [user_id, first_name, surname, Gender, state, Age, date_joined, number_of_dependants, family_status, income]
Index: []

### No duplicate rows were found

### Checking data type for columns

In [12]:
df_cust.dtypes

user_id                  int64
first_name              object
surname                 object
Gender                  object
state                   object
Age                      int64
date_joined             object
number_of_dependants     int64
family_status           object
income                   int64
dtype: object

In [13]:
# Change the date_joined data type to datetime
df_cust['date_joined']=pd.to_datetime(df_cust['date_joined'])

In [14]:
df_cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   user_id               206209 non-null  int64         
 1   first_name            194950 non-null  object        
 2   surname               206209 non-null  object        
 3   Gender                206209 non-null  object        
 4   state                 206209 non-null  object        
 5   Age                   206209 non-null  int64         
 6   date_joined           206209 non-null  datetime64[ns]
 7   number_of_dependants  206209 non-null  int64         
 8   family_status         206209 non-null  object        
 9   income                206209 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 15.7+ MB


### Combine both dataframes

In [15]:
df_ords_prods.head()

product_id                product_name  aisle_id  department_id  prices  \
0           1  Chocolate Sandwich Cookies        61             19     5.8   
1           1  Chocolate Sandwich Cookies        61             19     5.8   
2           1  Chocolate Sandwich Cookies        61             19     5.8   
3           1  Chocolate Sandwich Cookies        61             19     5.8   
4           1  Chocolate Sandwich Cookies        61             19     5.8   

   order_id  user_id  order_number  orders_day_of_week  order_hour_of_day  \
0   3139998      138            28                   6                 11   
1   1977647      138            30                   6                 17   
2    389851      709             2                   0                 21   
3    652770      764             1                   3                 13   
4   1813452      764             3                   4                 17   

   ...        price_label     busiest_day    busiest_days  \
0  ...  Mid-range product  Regularly busy  Regularly busy   
1  ...  Mid-range product  Regularly busy  Regularly busy   
2  ...  Mid-range product     Busiest day    Busiest days   
3  ...  Mid-range product  Regularly busy    Slowest days   
4  ...  Mid-range product      Least busy    Slowest days   

  busiest_period_of_day max_order      loyalty_flag mean_product_price  \
0           Most orders        32  Regular customer           6.935811   
1        Average orders        32  Regular customer           6.935811   
2        Average orders         5      New customer           7.930208   
3           Most orders         3      New customer           4.972414   
4        Average orders         3      New customer           4.972414   

  spending_flag  median_days_between_orders    order_frequency  
0   Low spender                         8.0  Frequent customer  
1   Low spender                         8.0  Frequent customer  
2   Low spender                         8.0  Frequent customer  
3   Low spender                         9.0  Frequent customer  
4   Low spender                         9.0  Frequent customer  

[5 rows x 24 columns]

In [16]:
#Combining the dataframes using the merge() function
df_merged= df_ords_prods.merge(df_cust,on='user_id')

In [18]:
df_merged.head()

product_id                product_name  aisle_id  department_id  prices  \
0           1  Chocolate Sandwich Cookies        61             19     5.8   
1           1  Chocolate Sandwich Cookies        61             19     5.8   
2         907        Premium Sliced Bacon       106             12    20.0   
3         907        Premium Sliced Bacon       106             12    20.0   
4        1000                    Apricots        18             10    12.9   

   order_id  user_id  order_number  orders_day_of_week  order_hour_of_day  \
0   3139998      138            28                   6                 11   
1   1977647      138            30                   6                 17   
2   3160996      138             1                   5                 13   
3   2254091      138            10                   5                 14   
4    505689      138             9                   6                 12   

   ...    order_frequency  first_name  surname Gender      state Age  \
0  ...  Frequent customer     Charles      Cox   Male  Minnesota  81   
1  ...  Frequent customer     Charles      Cox   Male  Minnesota  81   
2  ...  Frequent customer     Charles      Cox   Male  Minnesota  81   
3  ...  Frequent customer     Charles      Cox   Male  Minnesota  81   
4  ...  Frequent customer     Charles      Cox   Male  Minnesota  81   

  date_joined number_of_dependants  family_status income  
0  2019-08-01                    1        married  49620  
1  2019-08-01                    1        married  49620  
2  2019-08-01                    1        married  49620  
3  2019-08-01                    1        married  49620  
4  2019-08-01                    1        married  49620  

[5 rows x 33 columns]

## Exporting dataframes

In [19]:
df_merged.to_pickle(os.path.join(path, '02 Data', 'Manipulated Data', 'ords_prods_cust_combined.pkl'))

In [20]:
df_merged.to_csv(os.path.join(path, '02 Data', 'Manipulated Data', 'ords_prods_cust_combined.csv'))